In [1]:
import netCDF4 as nc
import numpy as np
import time
import os
import xarray as xr
import matplotlib.pyplot as plt

C:\Users\zahu573\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [69]:
#made  a copy of surfdata_1720x1_sparse_grid_221202.nc with suffix update1k. This copied file would be updated with high res data
surfdata_sparse = nc.Dataset( 'C:/Users/zahu573/D/ELM/clm-netcdf/surfdata_1175x1_sparse_grid_American_230201_update1k.nc','r+')
ncid_highres = nc.Dataset('C:/Users/zahu573/D/ELM/clm-netcdf/surfdat_1k_pn_2017.nc','r')

In [14]:
surfdata_sparse_org = nc.Dataset( 'C:/Users/zahu573/D/ELM/clm-netcdf/surfdata_1175x1_sparse_grid_American_230201.nc','r')

In [4]:
lat_cmax = 47.72916667
lat_cmin = 47.4625
lon_cmax = 239.3708333
lon_cmin = 238.8625

In [5]:
latbounds = [ lat_cmin , lat_cmax ]
lonbounds = [ lon_cmin , lon_cmax ] # degrees east ? 
lats = ncid_highres.variables['lat'][:] 
lons = ncid_highres.variables['lon'][:]+360

# latitude lower and upper index
latli = np.argmin( np.abs( lats - latbounds[0] ) )
latui = np.argmin( np.abs( lats - latbounds[1] ) ) 

# longitude lower and upper index
lonli = np.argmin( np.abs( lons - lonbounds[0] ) )
lonui = np.argmin( np.abs( lons - lonbounds[1] ) )  
print(lonli,lonui, latli,latui)

463 524 895 927


In [3]:
'''Define variables'''
varnames = []
for ivar in ncid_highres.variables:
    varname = ncid_highres.variables[ivar].name
    xtype = ncid_highres.variables[ivar].dtype
    dimids = ncid_highres.variables[ivar].dimensions
    natts = len(ncid_highres.variables[ivar].ncattrs())
    varnames.append(varname)
    print(varname)

lat
lon
PCT_NAT_PFT_INDEX
PCT_CLAY
PCT_SAND
ORGANIC
TOPO
SLOPE
STD_ELEV
PCT_NATVEG
PCT_LAKE
PCT_WETLAND
PCT_GLACIER
PCT_URBAN
PCT_NAT_PFT
MONTHLY_HEIGHT_BOT
MONTHLY_HEIGHT_TOP
MONTHLY_LAI
MONTHLY_SAI


In [4]:
'''Find the nearest neighbor index within high res dataset'''
#Get Lat/Lon for global 2D grid.
for ivar in varnames:
    if ivar=='lat':
        y = ncid_highres.variables[ivar][:]
    elif ivar=='lon':
        x = ncid_highres.variables[ivar][:]+360

longxy,latixy = np.meshgrid(x,y)
#allocate memory
ii_idx = np.zeros_like(surfdata_sparse.variables['LATIXY'])  
jj_idx = np.zeros_like(surfdata_sparse.variables['LONGXY'])

In [5]:

#find the nearest index
for ii in range(np.shape(surfdata_sparse.variables['LONGXY'])[0]):
    dist = (longxy-surfdata_sparse.variables['LONGXY'][ii])**2 + (latixy - surfdata_sparse.variables['LATIXY'][ii])**2
    # nearest_cell_i_idx, nearest_cell_j_idx = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
    nearest_cell_i_idx, nearest_cell_j_idx = np.where((dist==dist.min()))
    print(nearest_cell_i_idx, nearest_cell_j_idx)
    if len(nearest_cell_i_idx) > 1:
        print('  WARNING: Site with (lat,lon) = (%s,%s) has more than one cells that are equidistant.' %(str(lat_region[ii]),str(lon_region[ii])))
        print('Picking the first closest grid cell.')
    # print(dist.min())
    ii_idx[ii] = nearest_cell_i_idx[0]
    jj_idx[ii] = nearest_cell_j_idx[0]

ii_idx = ii_idx.astype(int)
jj_idx = jj_idx.astype(int)
print(ii_idx,jj_idx)

[816] [416]
[816] [417]
[816] [418]
[816] [419]
[816] [420]
[816] [421]
[816] [422]
[816] [423]
[816] [424]
[816] [425]
[816] [426]
[816] [427]
[816] [428]
[816] [429]
[816] [430]
[816] [431]
[816] [432]
[816] [433]
[816] [434]
[816] [435]
[816] [436]
[816] [437]
[816] [438]
[816] [439]
[816] [440]
[816] [441]
[816] [442]
[816] [443]
[816] [444]
[816] [445]
[816] [446]
[816] [447]
[816] [448]
[816] [449]
[816] [450]
[816] [451]
[816] [452]
[816] [453]
[816] [454]
[816] [455]
[816] [456]
[816] [457]
[816] [458]
[816] [459]
[816] [460]
[816] [461]
[816] [462]
[817] [416]
[817] [417]
[817] [418]
[817] [419]
[817] [420]
[817] [421]
[817] [422]
[817] [423]
[817] [424]
[817] [425]
[817] [426]
[817] [427]
[817] [428]
[817] [429]
[817] [430]
[817] [431]
[817] [432]
[817] [433]
[817] [434]
[817] [435]
[817] [436]
[817] [437]
[817] [438]
[817] [439]
[817] [440]
[817] [441]
[817] [442]
[817] [443]
[817] [444]
[817] [445]
[817] [446]
[817] [447]
[817] [448]
[817] [449]
[817] [450]
[817] [451]
[817

In [6]:
'''Converts a 2D data (lat,lon) to 1D (gridcell)'''
def sgrid_convert_2d_to_1d(vardimids, ii_idx, jj_idx, data):
    data_2d = np.zeros(np.shape(ii_idx))
    for ii in range(np.shape(ii_idx)[0]):
        data_2d[ii]= data[ii_idx[ii],jj_idx[ii]]
    # (lon,lat) --> (gridcell)
    vardimids_new =  []
    for x in vardimids:
        if x in ['lsmlat','lsmlon']:
            if x == 'lsmlat':
                vardimids_new.append('gridcell')
        else:
            vardimids_new.append(x)
    vardimids = tuple(vardimids_new)
    dims = np.shape(data_2d)
    if len(dims)>1:
        dims_new = [dims[0]*dims[1]]+[dims[k] for k in range(2,len(dims))]
        print("length of data_2d dim exceeds 1. Update the code")
    else:
        dims_new = dims
    data_1d = np.reshape(data_2d,dims_new)
    return data_1d

In [7]:
'''Converts a 3D data (lat,lon) to 2D (gridcell)'''

def sgrid_convert_3d_to_2d(vardimids, ii_idx, jj_idx, data):
    nx = np.shape(ii_idx)[0]
    try:
        ny = np.shape(jj_idx)[1]
    except:
        ny = 1
    try:
        nz = np.shape(data)[0] 
    except:
        nz = 1
    # print(nx,ny,nz)
    data_3d = np.zeros((nz,ny,nx))
    # print(data_3d)
    # print(np.shape(data_3d))
    for ii in range(0,nx):
        for jj in range(0,ny):
            data_3d[:,0,ii] = data[:,ii_idx[ii],jj_idx[ii]]
    vardimids_new =  []
    for x in vardimids:
        if x in ['lsmlat','lsmlon']:
            if x == 'lsmlat':
                vardimids_new.append('gridcell')
        else:
            vardimids_new.append(x)
    vardimids = tuple(vardimids_new)
    # print(vardimids)
    dims = np.shape(data_3d)
    # print(dims)
    if len(dims)>2:
        dims_new = [dims[k] for k in range(0,len(dims)-2)]+[dims[1]*dims[2]]
        # print(dims_new)
    else:
        dims_new = [dims[1]*dims[2]]
    data_2d = np.reshape(data_3d,dims_new)
    
    return data_2d 

In [8]:
'''Converts a 4D data (lat,lon) to 3D (gridcell)'''

def sgrid_convert_4d_to_3d(vardimids, ii_idx, jj_idx, data):
    nx = np.shape(ii_idx)[0]
    try:
        ny = np.shape(ii_idx)[1]
    except:
        ny = 1
    nz = np.shape(data)[1]
    na = np.shape(data)[0]
    # print(nx, ny, nz,na)
    data_4d = np.zeros((na,nz,ny,nx))
    for ii in range(0,nx):
        for jj in range(0,ny):
            data_4d[:,:,0,ii] = data[:,:,ii_idx[ii],jj_idx[ii]]
    vardimids_new =  []
    for x in vardimids:
        if x in ['lsmlat','lsmlon']:
            if x == 'lsmlat':
                vardimids_new.append('gridcell')
        else:
            vardimids_new.append(x)
    vardimids = tuple(vardimids_new)
    dims = np.shape(data_4d)
    # print(vardimids,dims)
    if len(dims)>2:
        dims_new = [dims[k] for k in range(0,len(dims)-2)]+[dims[-1]*dims[-2]]
        # print(dims_new)
    else:
        dims_new = [dims[-1]*dims[-2]]
    data_3d = np.reshape(data_4d,dims_new)
    return data_3d

In [9]:
#copy variable values from high res 2D dataset to the sparse dataset
lonlat_found=1
for ivar in ncid_highres.variables:
    if ivar in surfdata_sparse.variables.keys():
        data = ncid_highres.variables[ivar][:]
        varname = ncid_highres.variables[ivar].name
        vartype = ncid_highres.variables[ivar].dtype
        vardimids = ncid_highres.variables[ivar].dimensions
        varnatts = len(ncid_highres.variables[ivar].ncattrs())
        print(varname, vardimids )
        if len(vardimids)==2:
            print('vardimids length 2',varname)
            if 'lat' in vardimids:
                if lonlat_found==1:
                    # print((varname, vardimids,np.shape(data)))
                    data_1d = sgrid_convert_2d_to_1d(vardimids, ii_idx, jj_idx, data)
                    data_new=data_1d
                else:
                    data_2d = ugrid_convert_2d_to_2d(ii_idx, data)
                surfdata_sparse[ivar][:] = data_new
            else:
                # print('else case 2')
                surfdata_sparse[ivar][:] = data

        elif len(vardimids)==3:
            print('vardimids length 3',varname)
            if 'lat' in vardimids:
                if lonlat_found==1:
                    # print(varname, vardimids,np.shape(data))
                    data_2d  = sgrid_convert_3d_to_2d(vardimids, ii_idx, jj_idx, data)
                    data_new = data_2d
                    surfdata_sparse[ivar][:] = data_new        
                else:
                    # print('case 3 else')
                    data_3d  = ugrid_convert_3d_to_3d(ii_idx, data)
                    data_new = PerformFractionCoverCheck(varname, data_3d, set_natural_veg_frac_to_one)
                    surfdata_sparse[ivar][:] = data_new 
            else:
                # print('no latlon')
                ncid_out[ivar][:] = data                    
        elif len(vardimids)==4:
            print('vardimids length 4',varname)
            if 'lat' in vardimids:
                if lonlat_found==1:
                    # print(varname,np.shape(data))
                    data_3d = sgrid_convert_4d_to_3d(vardimids, ii_idx, jj_idx, data)
                else:
                    print('error')
                surfdata_sparse[ivar][:] = data_3d 
            else:
                print('else vardimids length 4')
                surfdata_sparse[ivar][:] = data 
        else:
            print('error')

PCT_CLAY ('nlevsoi', 'lat', 'lon')
vardimids length 3 PCT_CLAY
PCT_SAND ('nlevsoi', 'lat', 'lon')
vardimids length 3 PCT_SAND
ORGANIC ('nlevsoi', 'lat', 'lon')
vardimids length 3 ORGANIC
TOPO ('lat', 'lon')
vardimids length 2 TOPO
SLOPE ('lat', 'lon')
vardimids length 2 SLOPE
STD_ELEV ('lat', 'lon')
vardimids length 2 STD_ELEV
PCT_NATVEG ('lat', 'lon')
vardimids length 2 PCT_NATVEG
PCT_LAKE ('lat', 'lon')
vardimids length 2 PCT_LAKE
PCT_WETLAND ('lat', 'lon')
vardimids length 2 PCT_WETLAND
PCT_GLACIER ('lat', 'lon')
vardimids length 2 PCT_GLACIER
PCT_URBAN ('density_class', 'lat', 'lon')
vardimids length 3 PCT_URBAN
PCT_NAT_PFT ('pft', 'lat', 'lon')
vardimids length 3 PCT_NAT_PFT
MONTHLY_HEIGHT_BOT ('time', 'pft', 'lat', 'lon')
vardimids length 4 MONTHLY_HEIGHT_BOT
MONTHLY_HEIGHT_TOP ('time', 'pft', 'lat', 'lon')
vardimids length 4 MONTHLY_HEIGHT_TOP
MONTHLY_LAI ('time', 'pft', 'lat', 'lon')
vardimids length 4 MONTHLY_LAI
MONTHLY_SAI ('time', 'pft', 'lat', 'lon')
vardimids length 4 MON

### Check if variables were updated with correct values

In [49]:
different =[]
var='PCT_GLACIER'
for i in range(np.shape(surfdata_sparse.variables['LONGXY'])[0]):
    # print(surfdata_sparse.variables['LATIXY'][i],surfdata_sparse.variables['LONGXY'][i],surfdata_sparse_org.variables['LATIXY'][i],surfdata_sparse_org.variables['LONGXY'][i],ncid_highres['lat'][ii_idx[i]],ncid_highres['lon'][jj_idx[i]]+360)
    # print(surfdata_sparse_org.variables['MONTHLY_LAI'][7][2][i],surfdata_sparse.variables['MONTHLY_LAI'][7][2][i],ncid_highres.variables['MONTHLY_LAI'][7][2][ii_idx[i],jj_idx[i]])
    # if surfdata_sparse_org.variables['LATIXY'][i]-ncid_highres['lat'][ii_idx[i]]>abs(0.01) or surfdata_sparse_org.variables['LONGXY'][i]-(ncid_highres['lon'][jj_idx[i]]+360)>abs(0.01):
    #     print(surfdata_sparse.variables['LATIXY'][i],surfdata_sparse.variables['LONGXY'][i],surfdata_sparse_org.variables['LATIXY'][i],surfdata_sparse_org.variables['LONGXY'][i],ncid_highres['lat'][ii_idx[i]],ncid_highres['lon'][jj_idx[i]]+360)
    # for t in range(8,9):
    # for k in range (0,3):
    print(i)
    print(surfdata_sparse_org.variables[var][i],surfdata_sparse.variables[var][i],ncid_highres.variables[var][ii_idx[i],jj_idx[i]])
    if surfdata_sparse.variables[var][i]-ncid_highres.variables[var][ii_idx[i],jj_idx[i]]>abs(0.01):
        # print('Different',surfdata_sparse_org.variables['MONTHLY_LAI'][t][k][i],surfdata_sparse.variables['MONTHLY_LAI'][t][k][i],ncid_highres.variables['MONTHLY_LAI'][t][k][ii_idx[i],jj_idx[i]])
        different.append(surfdata_sparse.variables[var][i]-ncid_highres.variables[var][ii_idx[i],jj_idx[i]])

0
1.0 0.0 0.0
1
1.0 0.0 0.0
2
1.0 0.0 0.0
3
1.0 0.0 0.0
4
1.0 0.0 0.0
5
1.0 0.0 0.0
6
1.0 0.0 0.0
7
1.0 0.0 0.0
8
1.0 0.0 0.0
9
1.0 0.0 0.0
10
1.0 0.0 0.0
11
1.0 0.0 0.0
12
1.0 0.0 0.0
13
1.0 0.0 0.0
14
1.0 0.0 0.0
15
1.0 0.0 0.0
16
1.0 0.0 0.0
17
1.0 0.0 0.0
18
1.0 0.0 0.0
19
1.0 0.0 0.0
20
1.0 0.0 0.0
21
1.0 0.0 0.0
22
1.0 0.0 0.0
23
1.0 0.0 0.0
24
1.0 0.0 0.0
25
1.0 0.0 0.0
26
1.0 0.0 0.0
27
1.0 0.0 0.0
28
1.0 0.0 0.0
29
1.0 0.0 0.0
30
1.0 0.0 0.0
31
1.0 0.0 0.0
32
1.0 0.0 0.0
33
1.0 0.0 0.0
34
1.0 0.0 0.0
35
1.0 0.0 0.0
36
1.0 0.0 0.0
37
1.0 0.0 0.0
38
1.0 0.0 0.0
39
1.0 0.0 0.0
40
1.0 0.0 0.0
41
1.0 0.0 0.0
42
1.0 0.0 0.0
43
1.0 0.0 0.0
44
1.0 0.0 0.0
45
1.0 0.0 0.0
46
1.0 0.0 0.0
47
1.0 0.0 0.0
48
1.0 0.0 0.0
49
1.0 0.0 0.0
50
1.0 0.0 0.0
51
1.0 0.0 0.0
52
1.0 0.0 0.0
53
1.0 0.0 0.0
54
1.0 0.0 0.0
55
1.0 0.0 0.0
56
1.0 0.0 0.0
57
1.0 0.0 0.0
58
1.0 0.0 0.0
59
1.0 0.0 0.0
60
1.0 0.0 0.0
61
1.0 0.0 0.0
62
1.0 0.0 0.0
63
1.0 0.0 0.0
64
1.0 0.0 0.0
65
1.0 0.0 0.0
66
1.0 0.0 0.0
67
1.

In [10]:
surfdata_sparse.close()
# surfdata_sparse_org.close()